### MNIST classification

In [1]:
require 'nn';
require 'rnn';
matio = require 'matio'

In [2]:
data = matio.load('ex4data1.mat')
trainset = {}
trainset.data = data.X
trainset.label = data.y[{ {}, 1 }]

In [3]:
setmetatable(trainset,
    {__index = function(t,i)
                return {t.data[i], t.label[i]}
        end}
);
 
function trainset:size()
    return self.data:size(1)
end

In [4]:
mean = {}
stdv = {}
for i=1,400 do
    mean[i] = trainset.data[{ {},{i} }]:mean()
    stdv[i] = trainset.data[{ {}, {i} }]:std()
    --print(i .. 'th mean: ' .. mean[i])
    --print(i .. 'th std dev: ' .. stdv[i])
    trainset.data[{ {},{i} }]:add(-mean[i])
    if stdv[i] ~= 0 then
        trainset.data[{ {},{i} }]:div(stdv[i])
    end
end

### define model

In [5]:
batchSize = 5000
rho = 5
hiddenSize = 10
nIndex = 20
nClass = 10

In [6]:
rnn = nn.Sequential()
r = nn.Recurrent(
    hiddenSize, nn.Linear(nIndex, hiddenSize),
    nn.Linear(hiddenSize, hiddenSize), nn.Sigmoid(),
    rho
)
rnn:add(r)
rnn:add(nn.Linear(hiddenSize, nClass))
rnn:add(nn.LogSoftMax())
rnn = nn.Sequencer(rnn)

In [7]:
criterion = nn.SequencerCriterion(nn.ClassNLLCriterion())

### train

In [8]:
lr = 0.5
i = 1

In [10]:
prev = 100
for epoch = 1,1e3 do
    local inputs, targets = {}, {}
    for step=1,rho do
        table.insert(inputs, trainset.data[{{},{i, i+nIndex-1}}])
        table.insert(targets, trainset.label[{{}}])
        i = i+10
        if i+nIndex-1 > 400 then i = 1 end
    end
    local outputs = rnn:forward(inputs)
    local err = criterion:forward(outputs, targets)
    if epoch%10 == 1 then print(epoch, err/rho) end
    local gradOutputs = criterion:backward(outputs, targets)
    rnn:backward(inputs, gradOutputs)
    rnn:updateParameters(lr)
    rnn:zeroGradParameters()
    if prev<err and lr > 0.01 then 
        print("prev: ", prev, "cur: ", err, "lr", lr, "->", lr*0.5)
        lr = lr * 0.5 
    end
    prev = err
end

1	2.352516662625	


prev: 	11.420455846939	cur: 	11.468608233371	lr	0.5	->	0.25	


prev: 	11.397435659797	cur: 	11.43195139394	lr	0.25	->	0.125	


prev: 	11.43195139394	cur: 	11.508985897092	lr	0.125	->	0.0625	


prev: 	11.508985897092	cur: 	11.511550090343	lr	0.0625	->	0.03125	


...

951	2.2504398732509	


961	2.2800756917872	


971	2.1345242852888	


981	2.164336188607	


991	2.3055318719108	


### test

In [12]:
correction = {}
for i=20,38 do
    correction[i] = 0
end
for i=1,trainset:size() do
    local answer = trainset.label[i]
    local inputs = {}
    for step=0,38 do
        table.insert(inputs, trainset.data[{{i},{step*10+1, step*10+20}}])
    end
    local prediction = rnn:forward(inputs)
    for d=20,38 do
        guess = prediction[d][{1,{}}]
        local confidences, indices = torch.sort(guess, true)
        -- if i%100 == 1 then print(answer, guess, indices[1]) end
        if (answer == indices[1]) then
            correction[d] = correction[d] + 1
        end
    end
end
for i=20,38 do
    print(i, " = ", correction[i], 100*correction[i]/trainset:size() .. '%')
end

20	 = 	

948	18.96%	
21	 = 	887	17.74%	
22	 = 	803	16.06%	
23	 = 	757	15.14%	
24	 = 	810	16.2%	
25	 = 	1013	20.26%	
26	 = 	1100	22%	
27	 = 	1202	24.04%	
28	 = 	1235	24.7%	
29	 = 	1256	25.12%	
30	 = 	1221	24.42%	
31	 = 	1139	22.78%	
32	 = 	1020	20.4%	
33	 = 	954	19.08%	
34	 = 	768	15.36%	
35	 = 	712	14.24%	
36	 = 	650	13%	
37	 = 	534	10.68%	
38	 = 	561	11.22%	
